In [1]:
import pandas as pd

In [3]:
fifteenData = pd.read_csv("world_happiness/2015.csv")
sixteenData = pd.read_csv("world_happiness/2015.csv")
fifteenData = pd.read_csv("world_happiness/2015.csv")
fifteenData = pd.read_csv("world_happiness/2015.csv")
fifteenData = pd.read_csv("world_happiness/2015.csv")
